In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "5"
import os.path as path
from copy import copy

from joblib import Parallel, delayed

from ExperimentConfig import ExperimentConfig
from Experiment import Experiment
from CombinedExperiment import CombinedExperiment
from typing import List
from Experiment import RaceResult

class ComparisonResult:
    def __init__(self, results: List[RaceResult]):
        self.results = results

base_config = ExperimentConfig("base_config",
                         "IRV",
                         equal_pct_bins=True,
                         candidate_variance=.5,
                         quality_variance=0,
                         ideology_flexibility = .7,
                         sampling_voters=1000,
                         model_path=f"exp/v26/IRV-A.mdl.010000.progress")

def run_strategic_races(irv_config: ExperimentConfig, h2h_config: ExperimentConfig):
    candidates = base_config.gen_candidates(5)
    voters = irv_config.population.generate_unit_voters(irv_config.sampling_voters)
    def run_config(c: ExperimentConfig) -> RaceResult:
        c.election_config.uncertainty = 0
        x = Experiment(c)
        return x.run_strategic_race_c(candidates, voters)

    return [run_config(c) for c in [irv_config, h2h_config]]

def compute_SUE_single(rr: RaceResult):
    return CombinedExperiment.compute_SUE_single(rr)

import matplotlib.pyplot as plt
def make_line_plot(data, title, labels,
                   xlabel: str = "Candidate Ideological Flexibility (stddev)",
                   ylabel: str = "Social Utility Efficiency"):
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300
    n_rows = 1
    n_cols = 1
    fig, axis = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 10))
    fig.suptitle(title, color="black", fontsize=22)
    fig.set_facecolor("white")

    count = 0
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    axis.tick_params(axis='x', colors="black")

    for i in range(len(data)):
        d = data[i]
        plt.plot(d[0], d[1], label =labels[i])
    plt.legend()

    axis.set_xlabel(xlabel, fontsize=20)
    axis.set_ylabel(ylabel, fontsize=20)

import pickle
def save_results(file_path: str, obj):
    with open(file_path, "wb") as f:
        pickle.dump(obj, f)

def load_results(file_path: str):
    if path.exists(file_path):
        with open(file_path, "rb") as f:
            return pickle.load(f)
    else:
        return None

In [17]:
iterations = 50000
n_voters = 10000
irv_config = copy(base_config)
irv_config.election_name = "IRV"
irv_config.model_path = "exp/v26/IRV-A.mdl.%06d.progress" % (iterations )
irv_config.sampling_voters = 1000

h2h_config = copy(base_config)
h2h_config.election_name="H2H"
h2h_config.model_path = "exp/v26/H2H-A.mdl.%06d.progress" % (iterations )
h2h_config.sampling_voters = 1000

n_races = 1000
results: List[List[RaceResult]] = Parallel(n_jobs=32)(
    delayed(run_strategic_races)(irv_config, h2h_config) for _ in range(n_races))
# results = [run_strategic_races(irv_config, h2h_config) for _ in range(n_races)]


irv_count = 0
h2h_count = 0
equal_count = 0

for rr in results:
    irv_sue = compute_SUE_single(rr[0])
    h2h_sue = compute_SUE_single(rr[1])
    if irv_sue > h2h_sue:
        irv_count += 1
    elif h2h_sue > irv_sue:
        h2h_count += 1
    else:
        equal_count += 1

print("irv_count %d h2h_count %d equal_count %d" % (irv_count, h2h_count, equal_count))

irv_count 17 h2h_count 983 equal_count 0
